In [2]:
import pandas as pd
import numpy as np
import sklearn
from keras.models import Sequential
from keras.layers import Dense, Dropout, Masking, Embedding
from matplotlib import pyplot as plt
from ipywidgets import interact
import pickle
import math
from IPython.display import clear_output
plt.rcParams['figure.dpi'] = 200

ModuleNotFoundError: No module named 'tensorflow'

# Data In

In [ ]:
all_df_norm = pd.read_csv("data/all_df_basic_norm.csv")
norm_bounds = pd.read_csv("data/norm_bounds_basic.csv")
all_df = pd.read_csv("data/all_df_basic.csv")
all_df_shuffled, all_df_norm_shuffled = sklearn.utils.shuffle(all_df, all_df_norm)

### Data Segmenter

In [ ]:
def segment_data(data, x_labels, y_labels, split_factor=0.8):
    x_data = data[x_labels]
    y_data = data[y_labels]

    x_dim = len(x_labels)
    y_dim = len(y_labels)

    split = int(split_factor * len(x_data))

    x_train = x_data[:split]
    y_train = y_data[:split]

    x_test = x_data[split:]
    y_test = y_data[split:]
    return x_train, y_train, x_test, y_test, x_dim, y_dim

# Learned Forward Kinematics

### Data Labels

In [ ]:
fk_x_labels = ["M1-PL", "M1-PR", "M2-PL", "M2-PR",
            "M1-AL-IN-ACT", "M1-AR-IN-ACT", "M2-AL-IN-ACT", "M2-AR-IN-ACT",
            "M1-AL-OUT", "M1-AR-OUT", "M2-AL-OUT", "M2-AR-OUT"]
lookback = 3
new_x_labels = []
for i in range(lookback):
    for l in fk_x_labels:
        new_x_labels.append(l + str(i))
fk_x_labels += new_x_labels

fk_y_labels = ["M1X", "M1Y", "M2X", "M2Y", "M3X", "M3Y",
            "M4X", "M4Y", "M5X", "M5Y", "M6X", "M6Y",
            "M7X", "M7Y", "M8X", "M8Y", "M9X", "M9Y",
            "M10X", "M10Y"]

fk_x_train, fk_y_train, fk_x_test, fk_y_test, fk_x_dim, fk_y_dim = segment_data(all_df_norm_shuffled, fk_x_labels, fk_y_labels, split_factor=0.8)

### Model Structure

In [ ]:
fmodel = Sequential()
fmodel.add(Dense(128, input_dim=fk_x_dim, activation='relu'))
fmodel.add(Dropout(0.2))
fmodel.add(Dense(64, activation='relu'))
fmodel.add(Dropout(0.2))
fmodel.add(Dense(32, activation='relu'))
fmodel.add(Dropout(0.2))
fmodel.add(Dense(16, activation='relu'))
fmodel.add(Dropout(0.2))
fmodel.add(Dense(y_dim, activation='relu'))

fmodel.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
fmodel.summary()

In [ ]:
historyf = fmodel.fit(fk_x_train, fk_y_train, epochs=100, batch_size=8)

In [ ]:
loss, accuracy = fmodel.evaluate(fk_x_test, fk_y_test)
print('Loss: %.8f, Accuracy: %.2f' % (loss, (accuracy*100)))

# Learned Inverse Kinematics

### Data Labels

In [ ]:

ik_x_labels = ["M1X", "M1Y", "M2X", "M2Y", "M3X", "M3Y",
            "M4X", "M4Y", "M5X", "M5Y", "M6X", "M6Y",
            "M7X", "M7Y", "M8X", "M8Y", "M9X", "M9Y",
            "M10X", "M10Y"]
ik_y_labels = ["M1-PL", "M1-PR", "M2-PL", "M2-PR"]

ik_x_train, ik_y_train, ik_x_test, ik_y_test, ik_x_dim, ik_y_dim = segment_data(all_df_norm_shuffled, ik_x_labels, ik_y_labels, split_factor=0.8)

### Model Structure

In [ ]:
bmodel = Sequential()
bmodel.add(Dense(128, input_dim=ik_x_dim, activation='relu'))
bmodel.add(Dropout(0.2))
bmodel.add(Dense(64, activation='relu'))
bmodel.add(Dropout(0.2))
bmodel.add(Dense(32, activation='relu'))
bmodel.add(Dropout(0.2))
bmodel.add(Dense(16, activation='relu'))
bmodel.add(Dropout(0.2))
bmodel.add(Dense(y_dim, activation='relu'))

bmodel.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
bmodel.summary()

In [ ]:
historyb = bmodel.fit(ik_x_train, ik_y_train, epochs=100, batch_size=8)

In [ ]:
loss, accuracy = bmodel.evaluate(ik_x_test, ik_y_test)
print('Loss: %.8f, Accuracy: %.2f' % (loss, (accuracy*100)))

# Plots and Analysis

In [ ]:
def history_plot(historyf, historyb):
    plt.plot(historyf.history["loss"], label="Forward Kinematics Model")
    plt.plot(historyb.history["loss"], label="Inverse Kinematics Model")
    plt.legend()
    plt.xlabel("Epoch #")
    plt.ylabel("Loss (mse)")
    plt.title("Training Loss for Kinematics Models")
    # plt.savefig("data/training_backward.png")
history_plot(historyf, historyb)

In [ ]:
def rescale(data, labels):
    ndata = []
    for idx, label in enumerate(labels):
        d = data[idx]
        min_val = norm_bounds.loc[0, label]
        max_val = norm_bounds.loc[1, label]
        rescaled_d = (d * (max_val - min_val)) + min_val
        ndata.append(rescaled_d)
    return np.array(ndata)

In [ ]:
num_tests = len(x_test)
def evaluate_markers(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = fmodel.predict(x_select)
    cp = rescale(y_pred[0], y_labels)
    ct = all_df.loc[index, y_labels].to_numpy()
    
    xp = [0]
    xt = [0]
    yp = [0]
    yt = [0]
    
    for i in range(len(cp)):
        if i%2 == 0:
            xp.append(cp[i])
            xt.append(ct[i])
        else:
            yp.append(cp[i])
            yt.append(ct[i])
    
    plt.plot(xt, yt, "-o", label="actual")
    plt.plot(xp, yp, "-o", label="predicted")
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.xlabel("x (cm)")
    plt.ylabel("y (cm)")
    plt.legend()
    plt.title("Predicted Markers vs. Actual Markers")
    plt.show()
        
interact(evaluate_markers)

In [ ]:
bmodel.save("/Volumes/Flash/basic_bmodel/")

In [ ]:
num_tests = len(x_test)
def evaluate_markers(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    print(x_select)
    print(x_select.shape)
    y_pred = bmodel.predict(x_select)
    pp = rescale(y_pred[0], y_labels)
    pt = all_df.loc[index, y_labels].to_numpy()
    print(pp)
    print(pt)
    plt.plot([1,2], pt[0:2], "-o", label="actual M1", color="C0")
    plt.plot([3,4], pt[2:4], "-x", label="actual M2", color="C0")
    plt.plot([1,2], pp[0:2], "-o", label="predicted M1", color="C1")
    plt.plot([3,4], pp[2:4], "-x", label="predicted M2", color="C1")
    plt.ylim([95,120])
    plt.xlim([0.5,4.5])
    plt.xticks([1,2,3,4], ["P_1,1", "P_1,2", "P_2,1", "P_2,2"])
    plt.legend()
    plt.xlabel("Pressure Sensor")
    plt.ylabel("Pressure Value (kPa)")
    plt.show()
        
interact(evaluate_markers)

In [ ]:
num_tests = len(x_test)
def evaluate_polynomial(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    p_pred = pmodel.predict(x_select)
    pp = rescale(p_pred[0], p_labels)
    pt = all_df.loc[index, p_labels].to_numpy()
    
    ap = np.flip(pp[:-1])
    dp = pp[-1]
    at = np.flip(pt[:-1])
    dt = pt[-1]
    
    polyp = np.poly1d(ap)
    polyt = np.poly1d(at)
    
    yp = np.linspace(0, dp)
    xp = polyp(yp)
    
    yt = np.linspace(0, dt)
    xt = polyt(yt)
    
    
    plt.plot(xt, yt, label="actual")
    plt.plot(xp, yp, label="predicted")
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.xlabel("x (cm)")
    plt.ylabel("y (cm)")
    plt.legend()
    plt.title("Predicted Markers vs. Actual Markers")
    plt.show()
        
interact(evaluate_polynomial)

In [ ]:
errors = []
for idx in range(len(x_test)):
    index = x_test.index[idx]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = bmodel.predict(x_select)
    y_pred = rescale(y_pred[0], y_labels)
    y_select = all_df.loc[index, y_labels].to_numpy()
    diff = (y_pred - y_select)
    mse = (diff * diff).sum() / len(y_labels)
    rmse = math.sqrt(mse)
    errors.append((rmse, index))

In [ ]:
errors.sort()
print(errors)

In [ ]:
num_tests = len(x_test)
def evaluate_sort_markers(idx=(0,num_tests-1,1)):
    index = errors[idx][1]
    err = errors[idx][0]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = fmodel.predict(x_select)
    cp = rescale(y_pred[0], y_labels)
    ct = all_df.loc[index, y_labels].to_numpy()
    
    xp = [0]
    xt = [0]
    yp = [0]
    yt = [0]
    
    for i in range(len(cp)):
        if i%2 == 0:
            xp.append(cp[i])
            xt.append(ct[i])
        else:
            yp.append(cp[i])
            yt.append(ct[i])
    
    plt.plot(xt, yt, "-o", label="actual")
    plt.plot(xp, yp, "-o", label="predicted")
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.xlabel("x (cm)")
    plt.ylabel("y (cm)")
    plt.legend()
    plt.title("Predicted Markers vs. Actual Markers. RMSE: " + str(round(err,5)) + "cm")
    plt.show()
        
interact(evaluate_sort_markers)

In [ ]:
just_errors = []
for e in errors:
    just_errors.append(e[0])
plt.hist(just_errors, bins=30)
plt.title("Histogram of Root Mean Squared Error for Forward Model")
plt.ylim([0,170])
plt.ylabel("Number of Predictions")
plt.xlabel("Root Mean Squared Error (cm)")
plt.savefig("data/forward_error_histogram.png")

In [ ]:
file_index = 9
data_index = 570
for idx, e in enumerate(errors):
    index = errors[idx][1]
    file_index_temp = all_df.loc[index, "file_index"]
    data_index_temp = all_df.loc[index, "data_index"]
    if file_index_temp == file_index and data_index_temp == data_index:
        break
#idx = 1157
index = errors[idx][1]
err = errors[idx][0]
print(index)
x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
y_pred = fmodel.predict(x_select)
cp = rescale(y_pred[0], y_labels)
ct = all_df.loc[index, y_labels].to_numpy()

xp = [0]
xt = [0]
yp = [0]
yt = [0]

for i in range(len(cp)):
    if i%2 == 0:
        xp.append(cp[i])
        xt.append(ct[i])
    else:
        yp.append(cp[i])
        yt.append(ct[i])

plt.plot(xt, yt, "-o", label="actual")
plt.plot(xp, yp, "-o", label="predicted")
plt.xlim([-20,20])
plt.ylim([-1,39])
plt.xlabel("x (cm)")
plt.ylabel("y (cm)")
plt.legend()
plt.title("Predicted Markers vs. Actual Markers. RMSE: " + str(round(err,5)) + "cm")
pd.set_option("display.max_rows", None, "display.max_columns", None)
file_index = all_df.loc[index, "file_index"]
data_index = all_df.loc[index, "data_index"]
fname = "data/good" + str(file_index) + "_" + str(data_index) + "_2.png"
print(fname)
plt.savefig(fname)

In [ ]:
print(len(x_train))

In [ ]:
num_tests = len(x_test)
def evaluate_sort_pressures(idx=(0,num_tests-1,1)):
    index = errors[idx][1]
    err = errors[idx][0]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = bmodel.predict(x_select)
    pp = rescale(y_pred[0], y_labels)
    pt = all_df.loc[index, y_labels].to_numpy()
    print(pp)
    print(pt)
    plt.plot([1,2], pt[0:2], "-o", label="actual M1", color="C0")
    plt.plot([3,4], pt[2:4], "-x", label="actual M2", color="C0")
    plt.plot([1,2], pp[0:2], "-o", label="predicted M1", color="C1")
    plt.plot([3,4], pp[2:4], "-x", label="predicted M2", color="C1")
    plt.ylim([95,120])
    plt.xlim([0.5,4.5])
    plt.xticks([1,2,3,4], ["P_1,1", "P_1,2", "P_2,1", "P_2,2"])
    plt.xlabel("Pressure Sensor")
    plt.ylabel("Pressure Value (kPa)")
    plt.legend()
    plt.title("Predicted Pressures vs. Actual Pressures. RMSE: " + str(round(err,3)) + "kPa")
    plt.show()
        
interact(evaluate_sort_pressures)

In [ ]:
# file_index = 9
# data_index = 570
# for idx, e in enumerate(errors):
#     index = errors[idx][1]
#     file_index_temp = all_df.loc[index, "file_index"]
#     data_index_temp = all_df.loc[index, "data_index"]
#     if file_index_temp == file_index and data_index_temp == data_index:
#         break
idx = 1177
print(idx)
index = errors[idx][1]
err = errors[idx][0]
print(index)
x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
y_pred = bmodel.predict(x_select)
pp = rescale(y_pred[0], y_labels)
pt = all_df.loc[index, y_labels].to_numpy()
print(pp)
print(pt)
plt.plot([1,2], pt[0:2], "-o", label="actual M1", color="C0")
plt.plot([3,4], pt[2:4], "-x", label="actual M2", color="C0")
plt.plot([1,2], pp[0:2], "-o", label="predicted M1", color="C1")
plt.plot([3,4], pp[2:4], "-x", label="predicted M2", color="C1")
plt.ylim([95,120])
plt.xlim([0.5,4.5])
plt.xticks([1,2,3,4], ["P_1,1", "P_1,2", "P_2,1", "P_2,2"])
plt.xlabel("Pressure Sensor")
plt.ylabel("Pressure Value (kPa)")
plt.legend()
plt.title("Predicted Pressures vs. Actual Pressures. RMSE: " + str(round(err,3)) + "kPa")
pd.set_option("display.max_rows", None, "display.max_columns", None)
file_index = all_df.loc[index, "file_index"]
data_index = all_df.loc[index, "data_index"]
fname = "data_pressures/bad" + str(file_index) + "_" + str(data_index) + ".png"
print(fname)
plt.savefig(fname)

In [ ]:
just_errors = []
for e in errors:
    just_errors.append(e[0])
plt.hist(just_errors, bins=30)
plt.title("Histogram of Root Mean Squared Error for Backward Model")
plt.ylim([0,170])
plt.ylabel("Number of predictions")
plt.xlabel("Root Mean Squared Error (kPa)")
plt.savefig("data/backward_error_histogram.png")